In [1]:
import warnings
warnings.filterwarnings("ignore")

import datetime
import inspect
import io 
import json
import numpy as np
import pandas as pd
import requests 
from scipy.optimize import curve_fit


In [2]:
urlData = requests.get('https://corona.lmao.ninja/v2/historical/argentina').content

In [3]:
data = urlData.decode('utf-8')
data

'{"country":"Argentina","provinces":["mainland"],"timeline":{"cases":{"3/12/20":19,"3/13/20":31,"3/14/20":34,"3/15/20":45,"3/16/20":56,"3/17/20":68,"3/18/20":79,"3/19/20":97,"3/20/20":128,"3/21/20":158,"3/22/20":266,"3/23/20":301,"3/24/20":387,"3/25/20":387,"3/26/20":502,"3/27/20":589,"3/28/20":690,"3/29/20":745,"3/30/20":820,"3/31/20":1054,"4/1/20":1054,"4/2/20":1133,"4/3/20":1265,"4/4/20":1451,"4/5/20":1451,"4/6/20":1554,"4/7/20":1628,"4/8/20":1715,"4/9/20":1795,"4/10/20":1975},"deaths":{"3/12/20":1,"3/13/20":2,"3/14/20":2,"3/15/20":2,"3/16/20":2,"3/17/20":2,"3/18/20":2,"3/19/20":3,"3/20/20":3,"3/21/20":4,"3/22/20":4,"3/23/20":4,"3/24/20":6,"3/25/20":8,"3/26/20":9,"3/27/20":13,"3/28/20":18,"3/29/20":19,"3/30/20":23,"3/31/20":27,"4/1/20":28,"4/2/20":36,"4/3/20":39,"4/4/20":43,"4/5/20":44,"4/6/20":48,"4/7/20":56,"4/8/20":63,"4/9/20":72,"4/10/20":82},"recovered":{"3/12/20":0,"3/13/20":0,"3/14/20":1,"3/15/20":1,"3/16/20":1,"3/17/20":3,"3/18/20":3,"3/19/20":3,"3/20/20":3,"3/21/20":3,"3/22

In [4]:
data_load = json.loads(data)
data_load

{'country': 'Argentina',
 'provinces': ['mainland'],
 'timeline': {'cases': {'3/12/20': 19,
   '3/13/20': 31,
   '3/14/20': 34,
   '3/15/20': 45,
   '3/16/20': 56,
   '3/17/20': 68,
   '3/18/20': 79,
   '3/19/20': 97,
   '3/20/20': 128,
   '3/21/20': 158,
   '3/22/20': 266,
   '3/23/20': 301,
   '3/24/20': 387,
   '3/25/20': 387,
   '3/26/20': 502,
   '3/27/20': 589,
   '3/28/20': 690,
   '3/29/20': 745,
   '3/30/20': 820,
   '3/31/20': 1054,
   '4/1/20': 1054,
   '4/2/20': 1133,
   '4/3/20': 1265,
   '4/4/20': 1451,
   '4/5/20': 1451,
   '4/6/20': 1554,
   '4/7/20': 1628,
   '4/8/20': 1715,
   '4/9/20': 1795,
   '4/10/20': 1975},
  'deaths': {'3/12/20': 1,
   '3/13/20': 2,
   '3/14/20': 2,
   '3/15/20': 2,
   '3/16/20': 2,
   '3/17/20': 2,
   '3/18/20': 2,
   '3/19/20': 3,
   '3/20/20': 3,
   '3/21/20': 4,
   '3/22/20': 4,
   '3/23/20': 4,
   '3/24/20': 6,
   '3/25/20': 8,
   '3/26/20': 9,
   '3/27/20': 13,
   '3/28/20': 18,
   '3/29/20': 19,
   '3/30/20': 23,
   '3/31/20': 27,
   '4/

In [5]:
df_timeline = pd.json_normalize(data_load['timeline']['cases']).T
df_deaths = pd.json_normalize(data_load['timeline']['deaths']).T
df_recovered = pd.json_normalize(data_load['timeline']['recovered']).T

In [6]:
df_timeline.columns = ['confirmed']
df_deaths.columns = ['deaths']
df_recovered.columns = ['recovered']

In [7]:
data_argentina = pd.concat([df_timeline, df_deaths, df_recovered], axis=1, sort=False)
data_argentina

,confirmed,deaths,recovered
3/12/20,19,1,0
3/13/20,31,2,0
3/14/20,34,2,1
3/15/20,45,2,1
3/16/20,56,2,1
3/17/20,68,2,3
3/18/20,79,2,3
3/19/20,97,3,3
3/20/20,128,3,3
3/21/20,158,4,3


In [8]:
data_argentina.index=pd.to_datetime(data_argentina.index.astype(str), format='%m/%d/%y')

In [9]:
data_argentina['num_id'] = np.arange(len(data_argentina) + 1)[1:]

In [10]:
data_argentina

,confirmed,deaths,recovered,num_id
2020-03-12,19,1,0,1
2020-03-13,31,2,0,2
2020-03-14,34,2,1,3
2020-03-15,45,2,1,4
2020-03-16,56,2,1,5
2020-03-17,68,2,3,6
2020-03-18,79,2,3,7
2020-03-19,97,3,3,8
2020-03-20,128,3,3,9
2020-03-21,158,4,3,10


In [11]:
predicted_days = 7
firstPredictionDay = data_argentina.index[-1] + datetime.timedelta(days = 1)

In [12]:
class Models:    
    def cuadratic_model(x, a, b, c):
        return abs(a*x**2 + b*x +c)
    
    def cubic_model(x, a, b, c, d):
        return abs(a*x**3 + b*x**2 +c*x + d)
    
    def fourth_pow_model(x, a, b, c, d, e):
        return abs(a*x**4 + b*x**3 +c*x**2 + d*x + e)
    
    def fiveth_pow_model(x, a, b, c, d, e, f):
        return abs(a*x**5 + b*x**4 +c*x**3 + d*x**2 + e*x + f)
    
    def sixh_pow_model(x, a, b, c, d, e, f, g):
        return abs(a*x**6 + b*x**5 +c*x**4 + d*x**3 + e*x**2 + f*x + g)
    
    def exp_model(x, a, b):
        return a * np.exp(b * x)
    
    def complex_cuadratic_model(x, a, b, c, n2):
        return abs(a*(x - n2)**2 + b*x + c)
    
    def complex_cubic_model(x, a, b, c, d, n3, n2):
        return abs(a*(x - n3)**3 + b*(x - n2)**2 +c*x + d)

In [13]:
def formatPrediction(firstPredictionDay, day, prediction):
    formatedDate = str(firstPredictionDay + datetime.timedelta(days = day))[0:10]
    formatedPrediction = str(int(prediction))
    return "|" + formatedDate + " | " + formatedPrediction + "|"

def printPrediction(model, popt, x, y):
    firstPredictionDayId = x.max() + 1
    print("--- Predictions ---")
    for day in range (0, predicted_days):
        predictionId = firstPredictionDayId + day
        predictionValue = model(predictionId, *popt)
        print(formatPrediction(firstPredictionDay, day, predictionValue))
    print("-------------------")

In [14]:
def printFuncionParameters(model, popt):
        print('\n', model.__name__)
        args = inspect.getfullargspec(model).args[1:]
        for arg, value in zip(args, popt):
            print( arg, " = ", value)

In [15]:
def predictData(model, x, y):
    popt, _ = curve_fit(model, x, y)
    printPrediction(model, popt, x, y)
    printFuncionParameters(model, popt)
    print()

In [16]:
def isModel(methodName):
    return not(methodName.startswith('__'))

for methodName in dir(Models):
    if(isModel(methodName)):
        print(methodName)
        predictData(getattr(Models, methodName), data_argentina.num_id, data_argentina.confirmed)

complex_cuadratic_model
--- Predictions ---
|2020-04-11 | 2142|
|2020-04-12 | 2275|
|2020-04-13 | 2412|
|2020-04-14 | 2553|
|2020-04-15 | 2698|
|2020-04-16 | 2847|
|2020-04-17 | 3000|
-------------------

 complex_cuadratic_model
a  =  1.9144039100875891
b  =  19.420961355441133
c  =  -100.31684440584276
n2  =  1.7266697403349023

complex_cubic_model
--- Predictions ---
|2020-04-11 | 2023|
|2020-04-12 | 2109|
|2020-04-13 | 2192|
|2020-04-14 | 2270|
|2020-04-15 | 2343|
|2020-04-16 | 2411|
|2020-04-17 | 2473|
-------------------

 complex_cubic_model
a  =  0.08072700926351824
b  =  -7.215949633491365
c  =  -108.62015974645142
d  =  586.3434217175367
n3  =  -5.699175797514143
n2  =  9.832885728904335

cuadratic_model
--- Predictions ---
|2020-04-11 | 2142|
|2020-04-12 | 2275|
|2020-04-13 | 2412|
|2020-04-14 | 2553|
|2020-04-15 | 2698|
|2020-04-16 | 2847|
|2020-04-17 | 3000|
-------------------

 cuadratic_model
a  =  1.914403904249966
b  =  12.809874857729346
c  =  -94.60935810276283

cub